In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data_path = Path('../results/supplements')
assert data_path.exists()

In [3]:
lower_bounds = pd.read_csv(data_path / "lower_bounds.csv", index_col=0)
upper_bounds = pd.read_csv(data_path / "upper_bounds.csv", index_col=0)
lower_bounds.sort_values("subsystem").to_csv(data_path / 'lower_bounds.csv')
upper_bounds.sort_values("subsystem").to_csv(data_path / "upper_bounds.csv")

In [4]:
fva_upper = pd.read_csv(data_path / "fluxes_fva_upper.csv", index_col=0)
fva_upper.sort_values("subsystem").to_csv(data_path / 'fluxes_fva_upper.csv')
fva_lower = pd.read_csv(data_path / "fluxes_fva_lower.csv", index_col=0)
fva_lower.sort_values("subsystem").to_csv(data_path / 'fluxes_fva_lower.csv')

In [5]:
fluxes = pd.read_csv(data_path / "fluxes_pfba.csv", index_col=0)
fluxes.sort_values("subsystem").to_csv(data_path / "fluxes_pfba.csv")

In [6]:
reactions = pd.read_csv('../external_data/mitocore/rxns_machine_readable.csv', index_col=0)
reactions.index = reactions.index.map(lambda s: s[2:])

In [7]:
nad_rxns = list(
    reactions[
        reactions.Equation.str.contains("nad_")
        | reactions.Equation.str.contains("nadh_")
    ].index
)
nadp_rxns = list(
    reactions[
        reactions.Equation.str.contains("nadp_")
        | reactions.Equation.str.contains("nadph_")
    ].index
)

In [8]:
cell_lines = list(lower_bounds.columns[:4])
file_names = ["293mitoparp_high", "hek293_high", "293mitoparp_low", "hek293_low"]
files = dict(zip(cell_lines, file_names))
files

{'mitoparp': '293mitoparp_high',
 '293': 'hek293_high',
 'mitoparp + low NAD': '293mitoparp_low',
 '293 + low NAD': 'hek293_low'}

In [9]:
for cell_line in cell_lines:
    paths = lower_bounds['subsystem']
    file_name = files[cell_line] + '.csv'
    lb = lower_bounds.loc[:, cell_line]
    ub = upper_bounds.loc[:, cell_line]
    fb_low = fva_lower.loc[:, cell_line]
    fb_up = fva_upper.loc[:, cell_line]
    fl = fluxes.loc[:, cell_line]
    df = pd.concat([fl, lb, ub, fb_low, fb_up, paths], axis=1)
    df.columns = ['flux_pfba', 'lower_bound', 'upper_bound', 'fva_lower', 'fva_upper', 'pathway']
    df['nad_dependent'] = False
    df['nadp_dependent'] = False
    nad_present = [r for r in nad_rxns if r in lb.index]
    nadp_present = [r for r in nadp_rxns if r in lb.index]
    df.loc[nad_present, 'nad_dependent'] = True
    df.loc[nadp_present, 'nadp_dependent'] = True
    df.sort_values('pathway').to_csv(data_path / file_name)